In [10]:
using Distributed
using Base.Threads


@everywhere begin
    using BenchmarkTools
    using ParallelTemperingMonteCarlo
end


  ** incremental compilation may be fatally broken for this module **



┌ Warning: Package ParallelTemperingMonteCarlo does not have Distributed in its dependencies:
│ - If you have ParallelTemperingMonteCarlo checked out for development and have
│   added Distributed as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ParallelTemperingMonteCarlo
└ Loading Distributed into ParallelTemperingMonteCarlo from project dependency, future warnings for ParallelTemperingMonteCarlo are suppressed.


In order to use a function of several variables in the pmap environment we require a curry function

In [11]:
@everywhere curry(f,y) = x -> f(x,y)
@everywhere add_xy(x,y) = x + y 


In [12]:
pmap(curry(add_xy,10), 1:5)


5-element Vector{Int64}:
 11
 12
 13
 14
 15

NB spawning processes is expensive and _not recommended at all_ for simple loops and functions.

# Sync macros

sync requires all tasks inside to complete before moving on, async moves right on along without waiting, 

In [13]:
@sync begin
    sleep(2)
    println("slept for two")

    @async begin 
        sleep(5)
        println("nice and rested")
    end
#the async wrapper skips straight to done
    println("done")
end

slept for two
done


nice and rested


We'll simulate a complex process with a 2 second sleep

In [14]:
function simtest(x)
    sleep(2)
    return x
end

simtest (generic function with 1 method)

In [15]:
@time begin 
    veccy = []
    for i = 1:10

        y = simtest(i)
        push!(veccy,y)
        
    end
    println(veccy)
end

@time begin 
    @sync for i = 1:10
        veccy = []
        @async begin 
            y = simtest(i)
            push!(veccy,y)
        end
    end
    println(veccy)
end

Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
 20.330085 seconds (309.83 k allocations: 16.636 MiB, 0.08% gc time, 1.53% compilation time)


Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  2.015827 seconds (2.56 k allocations: 129.685 KiB, 0.62% compilation time)


NB: Asynchronous tasks are not parallel, demonstrated above they do boost performance for large operations. Use @spawn not @async for parallel operations

Let's test a data unsafe operation

In [33]:
function printtest(x::Bool = true)
    println("begin")
    if x == true
        @sync for i=1:10
            Distributed.@spawn println("$i $i $i $i $i ")
        end
    else
        for i=1:10
            println("$i $i $i $i $i ")
        end
    end
    println("end")
end

printtest (generic function with 2 methods)

In [34]:
@time printtest()
println("done")

@time printtest(false)
println("done")
# @time begin 
#     println("begin")
#     @threads for i=1:10
#         println("$i $i $i $i $i ")
#     end
#     println("end")
# end

begin


1 1 1 1 1 
2 2 2 2 2 
3 3 3 3 3 
4 4 4 4 4 
5 5 5 5 5 
6 6 6 6 6 
7 7 7 7 7 
8 8 8 8 8 
9 9 9 9 9 
10 10 10 10 10 


end
  0.400992 seconds (210.65 k allocations: 11.385 MiB, 99.66% compilation time)
done
begin
1 1 1 1 1 
2 2 2 2 2 
3 3 3 3 3 
4 4 4 4 4 
5 5 5 5 5 
6 6 6 6 6 
7 7 7 7 7 
8 8 8 8 8 
9 9 9 9 9 
10 10 10 10 10 
end
  0.000624 seconds (419 allocations: 12.875 KiB)
done


# NB 
Categorically do not use @spawn where data ordering is relevant. Additionally, it is considered bad practice to parallelise any operation faster than 100 $\mu m$ for threads or 100ms for spawning as it does not increase speed enough to compensate for the time taken to spawn the operation.

Below we test specific functions for this purpose.

## Onwards, time-testing some functions

In [35]:
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 300000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

above we just define the 13 atom system, below we show that threading halves the time taken to complete one mc_step per trajectory.

In [20]:
@benchmark begin 
    for i in 1:mc_params.n_traj
        mc_step!(mc_states[i],pot,ensemble,1,0,0);
    end
end



BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  13.400 μs …   6.396 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     20.400 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   25.756 μs ± 128.136 μs  ┊ GC (mean ± σ):  1.91% ± 1.35%

  ▃    ▂█    ▆                                                  
  █▂▁▁▄██▂▂▂▃█▆▂▂▃▇▃▅▅▂▃▂▂▄▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  13.4 μs         Histogram: frequency by time         49.8 μs <

 Memory estimate: 11.09 KiB, allocs estimate: 98.

In [21]:
@benchmark begin 
    @threads for i in 1:mc_params.n_traj
        mc_step!(mc_states[i],pot,ensemble,1,0,0);
    end
end


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  27.100 μs …  25.266 ms  ┊ GC (min … max): 0.00% … 98.97%
 Time  (median):     53.500 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   72.984 μs ± 317.122 μs  ┊ GC (mean ± σ):  3.43% ±  0.99%

     ▂▆█▇▅▃                                                     
  ▂▂▄███████▇▇▆▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▂▂▂▂▂▁▂ ▃
  27.1 μs         Histogram: frequency by time          244 μs <

 Memory estimate: 13.95 KiB, allocs estimate: 107.

In [22]:
@benchmark begin 
    for i in 1:mc_params.n_traj
        x = MCRun.atom_displacement(mc_states[i].config.pos[2],mc_states[i].max_displ[1],mc_states[i].config.bc);
    end
end


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  12.600 μs …  4.158 ms  ┊ GC (min … max): 0.00% … 97.62%
 Time  (median):     16.400 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   21.325 μs ± 62.162 μs  ┊ GC (mean ± σ):  1.90% ±  0.98%

  ▆▄   ▂█▅▂▁ ▁▃▃▃▂ ▁▄▂▄▆▃▁▃▂ ▂▂▁                              ▂
  ███▇██████▇███████████████▇████▇▆▆▇▇▇▅▆▅▆▅▄▅▅▆▇▆▆▆▆▅▆▅▅▄▅▅▅ █
  12.6 μs      Histogram: log(frequency) by time        44 μs <

 Memory estimate: 8.09 KiB, allocs estimate: 290.

In [23]:
@benchmark begin 
    @threads for i in 1:mc_params.n_traj
        x = MCRun.atom_displacement(mc_states[i].config.pos[2],mc_states[i].max_displ[1],mc_states[i].config.bc);
    end
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  24.900 μs …  32.992 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     59.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   92.799 μs ± 505.959 μs  ┊ GC (mean ± σ):  2.50% ± 0.99%

     ▆█▇▄▂                                                      
  ▁▁▆██████▇▆▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  24.9 μs         Histogram: frequency by time          343 μs <

 Memory estimate: 10.95 KiB, allocs estimate: 299.

There is a speed-up in calculation for the atom_displacement step, but the greatest increase is to complete the metropolis condition as well, so threading works perfectly for the dimer by simply threading the loops. 

# Time to test the writing step

Let's test printing 32 (traj) iterations of 55(atoms) and time this with the @sync @sync vs without

In [24]:
t1 = @benchmark begin
    filetest = open("test.dat", "w+")
    
    @sync begin
    @async for i=1:32
         @async for j=1:55
            write(filetest,"$i $j \n")
        end
    end
end

close(filetest)
end

BenchmarkTools.Trial: 530 samples with 1 evaluation.
 Range (min … max):  6.418 ms … 52.703 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.797 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.422 ms ±  3.078 ms  ┊ GC (mean ± σ):  1.07% ± 5.27%

   ▂▂▇█▆▅▅▄▄▄▅▆▅▄▄▂▃▃▂ ▁                                      
  ████████████████████▇█▆▇▄▄▄▇▆▆▁▄▆▄▆▁▆▄▆▆▁▄▆▄▁▆▄▄▄▄▁▁▁▁▁▁▁▄ █
  6.42 ms      Histogram: log(frequency) by time     20.8 ms <

 Memory estimate: 649.47 KiB, allocs estimate: 12509.

In [25]:
t2 = @benchmark begin
    filetest = open("test2.dat", "w+")
    
    begin
    for i=1:32
        for j=1:55
            write(filetest,"$i $j \n")
        end
    end
end

    close(filetest)
end

BenchmarkTools.Trial: 560 samples with 1 evaluation.
 Range (min … max):  6.241 ms … 24.304 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.197 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.907 ms ±  2.314 ms  ┊ GC (mean ± σ):  1.06% ± 5.01%

      ▂█▄                                                     
  ▇▅▅██████▇▅▅▄▅▆▆▇▇▇▆▆▄▃▄▃▄▃▃▂▂▁▃▂▂▁▃▂▂▁▁▁▂▂▁▂▁▂▁▁▁▁▂▂▁▂▂▂▂ ▃
  6.24 ms        Histogram: frequency by time        18.3 ms <

 Memory estimate: 633.19 KiB, allocs estimate: 12331.

It's slower to use the sync async functionality, but the atom-invariance within trajectories may make @spawn or @threads more useful

In [26]:
t3 = @benchmark begin
    filetest = open("test3.dat", "w+")
   
   @sync begin

   @async for i=1:32
        @threads for j=1:55
           write(filetest,"$i $j \n")
       end
   end
end

close(filetest)
end

BenchmarkTools.Trial: 378 samples with 1 evaluation.
 Range (min … max):   6.879 ms … 99.789 ms  ┊ GC (min … max): 0.00% … 76.25%
 Time  (median):     12.394 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.195 ms ±  6.057 ms  ┊ GC (mean ± σ):  1.53% ±  3.92%

        ▁▃█▂ ▂ ▃▂▅▂▄▅▆▂                                        
  ▄▃▇▆██████▆██████████▇█▇█▇▆▅▃▄▄▅▆▄▅▃▃▁▁▄▃▁▁▄▁▃▃▁▁▃▃▃▃▁▃▁▃▁▄ ▄
  6.88 ms         Histogram: frequency by time        26.6 ms <

 Memory estimate: 760.95 KiB, allocs estimate: 13750.

In [36]:
t4 = @benchmark begin
    filetest = open("test4.dat", "w+")
   
   @sync begin

   @async for i=1:32
        for j=1:55
        Threads.@spawn write(filetest,"$i $j \n")
       end
   end
end

close(filetest)
end

BenchmarkTools.Trial: 474 samples with 1 evaluation.
 Range (min … max):   6.729 ms … 50.094 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     10.110 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.539 ms ±  3.559 ms  ┊ GC (mean ± σ):  3.12% ± 8.35%

      ▁▄█▄▆▇▃▁                                                 
  ▃▄███████████▆▄▃▂▃▁▁▂▁▁▁▁▁▂▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▂ ▃
  6.73 ms         Histogram: frequency by time        30.9 ms <

 Memory estimate: 1.50 MiB, allocs estimate: 22939.

Nope. Writing shouldn't be parallelised. This isn't advantageous. Atom moves on the otherhand benefit from this. Operations, and in particular for-loops for the storage steps are next on the agenda

Testing some Runner functionality

In [37]:
function test_things(x,mc_states)
    file = RuNNer.writeinit(pwd())
    if x == true
        @sync begin
        @threads for mc_state in mc_states
            writeconfig(file,mc_state.config,"Cu")
        end
        end
    elseif x == false
        for mc_state in mc_states
            writeconfig(file,mc_state.config,"Cu")
        end
    elseif x == 1
        for mc_state in mc_states
           Distributed.@spawn writeconfig(file,mc_state.config,"Cu")
        end
    end
    close(file)
end


test_things (generic function with 1 method)

In [38]:
@benchmark test_things(true,mc_states)

@benchmark test_things(1,mc_states)

@benchmark test_things(false,mc_states)

BenchmarkTools.Trial: 788 samples with 1 evaluation.
 Range (min … max):  4.828 ms … 17.895 ms  ┊ GC (min … max): 0.00% … 63.33%
 Time  (median):     5.756 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.318 ms ±  1.634 ms  ┊ GC (mean ± σ):  0.87% ±  4.40%

  ▅▄  ▇█                                                      
  ███▇███▅▇▅▅▄▄▅▅▆▅▅▃▄▃▃▃▃▂▃▂▁▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▂▁▂▂▁▂▁▂▁▂▁▂▂▁▂ ▃
  4.83 ms        Histogram: frequency by time        14.1 ms <

 Memory estimate: 664.17 KiB, allocs estimate: 5039.

As expected, significantly faster to not thread the writing step, what we can try is writing several files. Next the pmap formalism.

In [31]:
 @everywhere curry(f,y1,y2,y3,y4,y5,y6,y7,y8) = x -> f(x,y1,y2,y3,y4,y5,y6,y7,y8)
 